In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [ ]:
inaugural_speeches = pd.DataFrame(columns=["date", "name", "text", "link"])
inauglinks = []
base_url = "https://www.presidency.ucsb.edu/documents/"
# URL of the page to scrape
url = "https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/inaugural-addresses"

# Send a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the links on the page
links = soup.find_all("a")


for link in links:
    if "inaugural-address-" in str(link) or "the-presidents-inaugural-address" in str(link) or "https://www.presidency.ucsb.edu/documents/inaugural-address" in str(link):
        url = link.get("href")
        date = link.text
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        text = soup.find("div", class_="field-docs-content").find_all("p")
        text = [t.text for t in text]
        text = " ".join(text)
        links2 = soup.find_all("a")
        for link in links2:
            if "people" in str(link):
                person = str(link.get("href")).split("-")[-1]
        inaugural_speeches.loc[len(inaugural_speeches.index)] = [date, person, text, url]


inaugural_speeches

In [ ]:
inaugural_speeches['text'][1]

In [ ]:
inaugural_speeches = inaugural_speeches.drop(index=range(41))
inaugural_speeches


In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import string
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer

In [ ]:
inaugural_speeches['text_lower'] = ["".join(item for item in lst).lower() for lst in inaugural_speeches['text']]

In [ ]:
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stopwords = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def preprocessing(text, tokenizer, stopwords, lemmatizer):
    # Make everything in the df["Text"] column into a lower-case string
    #text = ["".join(item for item in lst).lower() for lst in text]

    # Tokenize
    tokens = tokenizer.tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    
    return tokens

In [ ]:
inaugural_speeches['tokens'] = inaugural_speeches['text_lower'].apply(lambda x: preprocessing(x, tokenizer, stopwords, lemmatizer))
inaugural_speeches

In [ ]:
inaugural_speeches.reset_index(drop=True, inplace=True)


In [ ]:
inaugural_speeches['tokens'][0]

In [ ]:
inaugural_speeches.to_csv('../data/inaugural_speeches.csv')